<a href="https://colab.research.google.com/github/Stephanie9606/Python_Multimodal_Deep_Learning/blob/main/binary_vgg_memes_TF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
import tensorflow as tf
from tensorflow.keras import applications
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model 
from tensorflow.keras.layers import Dense, Input, Dropout, Flatten

In [ ]:
# Set up the device for GPU usage
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [ ]:
# connect to google drive for files
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Data Pre-processing

In [ ]:
# unzip image file
!unzip /content/drive/MyDrive/memes_image.zip

In [ ]:
memesDF = pd.read_csv('/content/drive/MyDrive/top5_memes_tidy.tsv', sep='\t')

In [ ]:
memesDF = memesDF.iloc[:, 2:9]

In [ ]:
memesDF['AltText'] = memesDF['AltText'].astype(pd.StringDtype())
memesDF['MemeLabel'] = memesDF['MemeLabel'].astype(pd.StringDtype())
memesDF['CaptionText'] = memesDF['CaptionText'].astype(pd.StringDtype())
memesDF['HashId'] = memesDF['HashId'].astype(pd.StringDtype())
memesDF['ImageURL'] = memesDF['ImageURL'].astype(pd.StringDtype())
memesDF['ImagePath'] = memesDF['ImagePath'].astype(pd.StringDtype())
memesDF['Image'] = memesDF['Image'].astype(pd.StringDtype())

In [ ]:
# Only get two label for Binary Classification
meme2cDF = memesDF[(memesDF['MemeLabel'] == "Scared Cat") | (memesDF['MemeLabel'] == "Who Killed Hannibal")]

In [ ]:
meme2cDF.head(3)

,AltText,CaptionText,ImageURL,HashId,MemeLabel,ImagePath,Image
0,Coke in Regal Cinemas > Pepsi in Regal Cinemas...,WHO WANTS A PEPSI?,http://i.imgflip.com/3xa87f.jpg,b54073fe552aa582c57a1a39e9db3f550067775516b844...,Who Killed Hannibal,/content/memes_image/3xa87f.jpg,3xa87f.jpg
1,Who Killed Hannibal | NORTH KOREA PEOPLE WITH ...,NORTH KOREA PEOPLE WITH CORONAVIRUS WE HAVE 0...,http://i.imgflip.com/3v9nm3.jpg,55a37d7aac7c769f4d517ea20b50c8ffa8e545491e45c1...,Who Killed Hannibal,/content/memes_image/3v9nm3.jpg,3v9nm3.jpg
2,Who Killed Hannibal | MINECRAFT CAVE UPDATE MI...,MINECRAFT CAVE UPDATE MINECRAFT,http://i.imgflip.com/3xjml3.jpg,ba01cf5a250ad0d9dc2901c5250ddf0e49abc8fb8e0a80...,Who Killed Hannibal,/content/memes_image/3xjml3.jpg,3xjml3.jpg


In [ ]:
# train test split
from sklearn.model_selection import train_test_split

me_trainDF, me_testDF = train_test_split(meme2cDF, test_size=0.2, random_state=15)
me_trainDF.index

Int64Index([3025,  363,  222, 3160,  100,   51, 2855, 2651,  135, 2370,
            ...
            1047,  927,  749, 3115,  221,  943,  630, 3183, 2789, 3254],
           dtype='int64', length=1861)

# Load Pre-trained Model

In [ ]:
tf.random.set_seed(45)

# Load original model with pretrained weights from imagenet
base_model = applications.VGG16(weights="imagenet")

# Freeze base model
base_model.trainable = False

# Create new model on top
img_height = 224
img_width = 224
inputs = Input(shape=(img_height, img_width, 3))
x = base_model(inputs, training=False)
outputs = Dense(1, activation="softmax")(x) # output probabilities for both classes
model = Model(inputs, outputs)
model.summary()

553476096/553467096 [==============================] - 13s 0us/step
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 vgg16 (Functional)          (None, 1000)              138357544 
                                                                 
 dense (Dense)               (None, 1)                 1001      
                                                                 
Total params: 138,358,545
Trainable params: 1,001
Non-trainable params: 138,357,544
_________________________________________________________________


In [ ]:
# Try deeper model on top
img_height = 224
img_width = 224
inputs = Input(shape=(img_height, img_width, 3))
x = base_model(inputs, training=False)

hidden1 = Flatten(name="flatten")(x)
hidden2 = Dense(512, activation="relu")(hidden1)
hidden3 = Dropout(0.5)(hidden2)

outputs = Dense(1, activation="softmax")(hidden3) # we want to output probabilities for both classes
model = Model(inputs, outputs)
model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 vgg16 (Functional)          (None, 1000)              138357544 
                                                                 
 flatten (Flatten)           (None, 1000)              0         
                                                                 
 dense_1 (Dense)             (None, 512)               512512    
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                                 
 dense_2 (Dense)             (None, 1)                 513       
                                                                 
Total params: 138,870,569
Trainable params: 513,025
Non-tra

In [ ]:
!pip install mlflow

In [ ]:
import mlflow.tensorflow
from tensorflow.keras.optimizers import Adam

In [ ]:
# Check out the MLflow UI as this runs
mlflow.tensorflow.autolog(every_n_iter=2)

In [ ]:
# optimizer and metric
model.compile(loss="binary_crossentropy", optimizer=Adam(lr=0.005), metrics=["accuracy"]) 

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [ ]:
 # Loading training data
batch_size = 32
train_datagen = ImageDataGenerator(preprocessing_function=applications.vgg16.preprocess_input, rescale=1./255.,validation_split=0.25)
train_generator = train_datagen.flow_from_dataframe(dataframe=me_trainDF, 
                                                    directory=None, 
                                                    x_col="ImagePath", 
                                                    y_col="MemeLabel", 
                                                    subset="training", 
                                                    seed = 55, 
                                                    class_mode="binary", 
                                                    target_size=(img_height, img_width), 
                                                    batch_size=batch_size)

valid_generator = train_datagen.flow_from_dataframe(dataframe=me_trainDF, 
                                                    directory=None, 
                                                    x_col="ImagePath", 
                                                    y_col="MemeLabel", 
                                                    subset="validation", 
                                                    seed = 55, 
                                                    class_mode="binary", 
                                                    target_size=(img_height, img_width), 
                                                    batch_size=batch_size)

print(f"Class labels: {train_generator.class_indices}")

#step_size = train_generator.n//train_generator.batch_size

Found 1396 validated image filenames belonging to 2 classes.
Found 465 validated image filenames belonging to 2 classes.
Class labels: {'Scared Cat': 0, 'Who Killed Hannibal': 1}


In [ ]:
# Train the model 
step_size_train=train_generator.n//train_generator.batch_size
step_size_valid=valid_generator.n//valid_generator.batch_size

model.fit(train_generator,
          steps_per_epoch=step_size_train,
          validation_data=valid_generator,
          validation_steps=step_size_valid,
          epochs=10,
          verbose=2 # show: epoch 1/10
)

2022/03/13 21:02:47 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID 'c17598ac7eb5401eb67c281c7bfb3b5f', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current tensorflow workflow


Epoch 1/10
43/43 - 39s - loss: 0.4820 - accuracy: 0.5227 - val_loss: 0.1872 - val_accuracy: 0.5112 - 39s/epoch - 912ms/step
Epoch 2/10
43/43 - 20s - loss: 0.0763 - accuracy: 0.5227 - val_loss: 0.0245 - val_accuracy: 0.5156 - 20s/epoch - 455ms/step
Epoch 3/10
43/43 - 20s - loss: 0.0232 - accuracy: 0.5220 - val_loss: 0.0111 - val_accuracy: 0.5112 - 20s/epoch - 456ms/step
Epoch 4/10
43/43 - 20s - loss: 0.0140 - accuracy: 0.5183 - val_loss: 0.0074 - val_accuracy: 0.5089 - 20s/epoch - 455ms/step
Epoch 5/10
43/43 - 20s - loss: 0.0133 - accuracy: 0.5191 - val_loss: 0.0063 - val_accuracy: 0.5156 - 20s/epoch - 472ms/step
Epoch 6/10
43/43 - 20s - loss: 0.0104 - accuracy: 0.5213 - val_loss: 0.0045 - val_accuracy: 0.5112 - 20s/epoch - 458ms/step
Epoch 7/10
43/43 - 20s - loss: 0.0099 - accuracy: 0.5198 - val_loss: 0.0045 - val_accuracy: 0.5067 - 20s/epoch - 459ms/step
Epoch 8/10
43/43 - 25s - loss: 0.0097 - accuracy: 0.5183 - val_loss: 0.0036 - val_accuracy: 0.5179 - 25s/epoch - 578ms/step
Epoch 9/

2022/03/13 21:06:47 WARNING mlflow.utils.requirements_utils: Found jaxlib version (0.3.0+cuda11.cudnn805) contains a local version label (+cuda11.cudnn805). MLflow logged a pip requirement for this package as 'jaxlib==0.3.0' without the local version label to make it installable from PyPI. To specify pip requirements containing local version labels, please use `conda_env` or `pip_requirements`.


In [ ]:
# Evaluate model on test set
test_datagen = ImageDataGenerator(preprocessing_function=applications.vgg16.preprocess_input, rescale=1./255.)

In [ ]:
# Evaluate
batch_size = 32

test_generator = test_datagen.flow_from_dataframe(
  dataframe=me_testDF, 
  directory=None, 
  x_col="ImagePath", 
  y_col="MemeLabel", 
  class_mode="binary", 
  target_size=(img_height, img_width),
  shuffle=False,
  batch_size=batch_size
)

step_size_test = test_generator.n//test_generator.batch_size

eval_results = model.evaluate(test_generator, steps=step_size_test)
print(f"Loss: {eval_results[0]}. Accuracy: {eval_results[1]}")

Found 466 validated image filenames belonging to 2 classes.
14/14 [==============================] - 5s 361ms/step - loss: 0.0023 - accuracy: 0.5112
Loss: 0.002295874059200287. Accuracy: 0.5111607313156128
